In [1]:
import pandas as pd
import my_functions as utils
from unidecode import unidecode
from datetime import datetime
import ast
import importlib
import numpy as np
importlib.reload(utils)

<module 'my_functions' from '/home/arthur/google-drive/repos/DSBD/artigo/my_functions.py'>

In [2]:
input_usuario = {
    "area":80,
    "endereco":"Rua Maranhão, 1940, apt 24, Portão, Curitiba",
    "bairro":"portao",
    "quartos":3,
    "suites":0,
    "banheiros":1,
    "vagas_garagem":1,
    "mobiliado":"Não",
    "mobilia_planejada":"Não",
    "imovel_decorado":"Não",
    "sacada":"Não",
    "piscina":"Não",
    "academia":"Não",
    "churrasqueira":"Não",
    "salao_de_festas":"Sim",
    "espaco_coworking":"Não",
    "playground":"Sim",
    "lavanderia":"Sim"
}

input = pd.DataFrame([input_usuario])

In [3]:
def feature_engineering(data):
    # year
    data['year'] = datetime.today().year

    # month
    data['month'] = datetime.today().month

    # day
    data['day'] = datetime.today().day

    # week of day
    data['week_of_year'] = datetime.today().isocalendar().week

    return data


def get_dados_wikipedia(data):
    dados_curitiba = utils.get_infos_curitiba()

    ajustes = {
        'ecoville':'mossungue',
        'champagnat':'bigorrilho',
        'bigoriilho':'bigorrilho',
        'alto da rua xv': 'alto da xv',
        'novo mundo ': 'novo mundo',
        'jd. das americas': 'jardim das americas',
        'cic':'cidade industrial'
    }

    # ajustando nome dos bairros
    data['bairro'] = data['bairro'].replace(ajustes)
    data['bairro'] = data['bairro'].apply(lambda x: unidecode(x))

    # cruzando dados da wikipedia 
    data = pd.merge(data, dados_curitiba[['bairro', 'regiao', 'qtd_domicilios_particulares', 'renda_media_responsaveis_domicilio']], on = 'bairro', how ='left')
    data = data.dropna()

    # tratando dados da wikipedia
    # df_feature_engineering['area_bairro'] = df_feature_engineering['area_bairro'].astype('float64')
    # df_feature_engineering['qtd_homens'] = df_feature_engineering['qtd_homens'].astype('int64')
    # df_feature_engineering['qtd_mulheres'] = df_feature_engineering['qtd_mulheres'].astype('int64')
    # df_feature_engineering['total'] = df_feature_engineering['total'].astype('int64')
    data['qtd_domicilios_particulares'] = data['qtd_domicilios_particulares'].astype('int64')
    data['renda_media_responsaveis_domicilio'] = data['renda_media_responsaveis_domicilio'].astype('float64')

    return data

def coordenadas_geograficas(data):
    data['latitude'] = data['endereco'].apply(lambda x: utils.get_lat_long(utils.obter_lat_long(x), 'latitude'))
    data['longitude'] = data['endereco'].apply(lambda x: utils.get_lat_long(utils.obter_lat_long(x), 'longitude'))

    return data

In [4]:
input = get_dados_wikipedia(input)

In [5]:
input = feature_engineering(input)

In [6]:
input['latitude'] = input['endereco'].apply(lambda x: utils.get_lat_long(utils.obter_lat_long(x), 'latitude'))
input['longitude'] = input['endereco'].apply(lambda x: utils.get_lat_long(utils.obter_lat_long(x), 'longitude'))

In [7]:
input

,area,endereco,bairro,quartos,suites,banheiros,vagas_garagem,mobiliado,mobilia_planejada,imovel_decorado,...,lavanderia,regiao,qtd_domicilios_particulares,renda_media_responsaveis_domicilio,year,month,day,week_of_year,latitude,longitude
0,80,"Rua Maranhão, 1940, apt 24, Portão, Curitiba",portao,3,0,1,1,Não,Não,Não,...,Sim,Fazendinha-Portão,13183,1722.89,2025,4,15,16,-22.881466,-43.251343


In [8]:
dict_preparation = utils.load_picked_file(name='dict_data_preparation')
features_selected = utils.load_picked_file(name='features_selected')
best_model = utils.load_picked_file(name='best_model')

In [9]:
input_transformed = utils.preparacao_dos_dados(df=input, dict_preparation=dict_preparation, is_train=False)

In [10]:
input_transformed.T

,0
area,-0.142764
endereco,"Rua Maranhão, 1940, apt 24, Portão, Curitiba"
bairro,2567.3
quartos,1.159419
suites,-0.705671
banheiros,-0.243116
vagas_garagem,0.346376
mobiliado,0
mobilia_planejada,0
imovel_decorado,0


In [11]:
y_pred = best_model.predict(input_transformed[features_selected])

In [20]:
round((float(y_pred[0])),2 )

7.62

In [21]:
round(float(np.expm1(y_pred[0])), 2)

2035.13

In [232]:
input_transformed

,area,endereco,bairro,quartos,suites,banheiros,vagas_garagem,mobiliado,mobilia_planejada,imovel_decorado,...,latitude,longitude,year_sin,year_cos,month_sin,month_cos,day_sin,day_cos,week_of_year_sin,week_of_year_cos
0,-0.076131,"Rua Maranhão, 1940, apt 24, Portão, Curitiba",2558.07,1.21694,-0.683789,-0.263459,0.445781,0,0,0,...,74.040661,196.487308,5.712879e-13,-1.0,0.866025,-0.5,0.221922,0.975065,0.970942,-0.239316
